# dailymedi용 크롤러

In [9]:
# from bs4 import BeautifulSoup
# import requests
# from tqdm import tqdm

# # 11개 질병
# diseases = ['탈모', '비만', '고혈압', '고지혈증', '당뇨', '백내장', '치매', '비염', '위염', '치주질환', '치핵']
# # disease = diseases[0]

# # 각 질병마다의 기사 url을 urls에 모음
# urls = []
# for disease in tqdm(diseases):
#     parent_url = f'https://www.dailymedi.com/news/search.php?sch_sca=&sch_sca2=&sch_scope=&sch_scope2=&sch_scope3=&sdate=&edate=&stx={disease}'
#     res = requests.get(parent_url)
#     soup = BeautifulSoup(res.content, 'html.parser')
#     links = ['https://www.dailymedi.com'+a['href'] for div in soup.find_all('div', class_='sch_tit') if (a := div.find('a')) and 'href' in a.attrs]
#     urls += links
# # print(urls)

# urls = list(set(urls))
# # print('수집된 url의 수:', len(urls))

# # 모든 기사를 하나의 리스트에 담음.
# all_articles = []
# for url in tqdm(urls):
#     res = requests.get(url)
#     soup = BeautifulSoup(res.content, 'html.parser')
#     p_tags = soup.find_all('p')

#     article = ''
#     for p_tag in p_tags:
#         paragraph = p_tag.get_text()
#         article += paragraph
#     all_articles.append(article)

# # print(all_articles)

['\ufeff\ufeff\ufeff\ufeff\'기적의 비만약\'으로 불리는 \ufeff비만치료제 \'위고비\'(성분명 세마글루티드)가 비대면 진료를 통해 손 쉽게 처방되면서 오남용 문제가 도마 위에 올랐다.보건복지부는 비만치료제 등 일부 치료제를 비대면진료에서 제외하는 방안을 검토하겠다는 입장을 밝혔다.23일 국회 보건복지위원회 국정감사에서\xa0최근 한국에 출시된 비만 치료제 위고비 오남용 우려와 불법 유통 문제에 대해\xa0정부가 홍보와 감시를 강화해야 한다는 지적이 쏟아졌다.위고비는 노보노디스크가 개발한 인크레틴 기반 비만치료제로 의사 처방이 필요한 전문의약품이다.\xa0일론 머스크 등 유명 인사들이 복용을 했다고 알려지면서 화제가 됐고 지난 15일 국내 출시됐다.식약처는 위고비를 BMI(체질량지수) 30 이상이거나 BMI가 27 이상이면서 당뇨병·고혈압 등 대사질환과 같은 동반 질환이 있는 과체중 또는 비만 환자 등에 처방하도록 했다.하지만 단순히\xa0\'다이어트용\'으로 만연하게 위고비를 처방 받는 사례가 이어지고 있어 논란이 일고 있다.개혁신당 이주영 의원은 "지난번 식약처 국정감사에서 위고비 출시를 앞두고 불법 유통이나 과대광고, 부적절한 비대면진료에 대해 질의한 바 있는데 출시 이후 우려가 현실이 됐다"고 질타했다.이어 "위고비 출시 이후 불법는 물론 남용 사례가 나오고 있다"며 "식약처는 수요자 맞춤형 홍보를 하겠다고 했는데 식약처보다 인플루언서 영향력이 더 커보인다"고 말했다.그러면서 "부적절한 접근 자체를 제도적으로 어렵게 할 필요가 있다. 사후피임약은 논의를 거쳐 비대면진료에서 빠졌다"며 "비대면진료 항목 추가에 예민한 전문성이 있어야 한다"고 강조했다.더불어민주당 백혜련 의원도 비대면 진료 플랫폼 닥터나우를 언급하며 문제점을 지적했다.\xa0특히 진료 시작부터 처방까지 21초 밖에 걸리지 않는 점을 들며 심각성을 강조했다.그는 "비대면 진료 시 본인 확인도 제대로 하지 않고 기본적인 환자 상태도 물어보지 않는다"며 "이는 정상체중인

# 웹로더를 이용한 청크

In [1]:
from bs4 import BeautifulSoup
import requests
from langchain.document_loaders import WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from tqdm import tqdm
import tiktoken


# 11개 질병
diseases = ['탈모', '비만', '고혈압', '고지혈증', '당뇨', '백내장', '치매', '비염', '위염', '치주질환', '치핵']
# disease = diseases[0]

# 각 질병마다의 기사 url을 urls에 모음
urls = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
for disease in tqdm(diseases):
    parent_url = f'https://www.dailymedi.com/news/search.php?sch_sca=&sch_sca2=&sch_scope=&sch_scope2=&sch_scope3=&sdate=&edate=&stx={disease}'
    res = requests.get(parent_url, headers=headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    links = ['https://www.dailymedi.com'+a['href'] for div in soup.find_all('div', class_='sch_tit') if (a := div.find('a')) and 'href' in a.attrs]
    urls += links

# 토큰 단위로 청킹
tokenizer = tiktoken.get_encoding('cl100k_base')
def token_length(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function = token_length
)

urls = list(set(urls))

all_articles = []
for url in tqdm(urls):
    docs = WebBaseLoader(url).load_and_split(text_splitter)
    all_articles += docs

all_articles[0]

USER_AGENT environment variable not set, consider setting it to identify your requests.
100%|██████████| 92/92 [00:14<00:00,  6.23it/s]


Document(metadata={'source': 'https://www.dailymedi.com/news/news_view.php?wr_id=917206', 'title': '유트로핀펜주·시네츄라·레켐비·펜타닐', 'description': '\ufeff\ufeff\ufeff\ufeff[양보혜·구교윤 기자]\xa0유트로핀펜주를 비롯해 시네츄라,\xa0레켐비, 펜터민 및 에토미데이트 등.이는 국회 보건복지위원회가 지난 10일 진행한 식품의약품안전처 국정감사장을 뜨겁게 달군 의약품들이다. 화두가 된 이들 의약품을 통해 2024년 국감 이슈를 정리해봤다.\xa0먼저 유트로핀펜주는 국감장에 실제 등장했다. 김남희 더불어민주당 의원이 제품을 들고 나와 무분별하게 처방되고 있는 성장호르몬 주사에 대한 식약처의 부실한 관리 문제를 지적했다.\xa0김 의원은 ', 'language': 'ko'}, page_content='유트로핀펜주·시네츄라·레켐비·펜타닐\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n광고안내\n|회원가입\n|로그인\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n기사검색\n\n\n2024\n10.28\n월요일 \n\n\n\n\n\n\n\n\n\n\n\n\n\n뉴스\n\n\n행정/법률의원/병원학술/학회의대/전공의제약‧바이오의료기기/IT간호치과약국/유통월드뉴스한방e-談 \n\n\n\n\n\n\n\n\n\n\n\n\n\n금주의 검색어\n\n\n\n\n1\n코로나19\n\n2\nK-방역\n\n3\n대한의사협회\n\n4\n간호법\n\n5\n삼성서울병원\n \n\n\n\n\n6\n비대면\n\n7\n응급의료\n\n8\n종근당\n\n9\n재택치료\n\n10\n의료법인')

In [2]:
import json

dailymedi_news = []
for info in all_articles:
    dailymedi_news.append(
        {
            'page_content':info.page_content,
            'metadata':info.metadata
         }
    )

with open('dailymedi_news.json', 'w') as json_file:
    json.dump(dailymedi_news, json_file, indent=4)

In [4]:
with open('dailymedi_news.json', 'r') as f:
    data=json.load(f)
data[0]

{'page_content': '유트로핀펜주·시네츄라·레켐비·펜타닐\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n광고안내\n|회원가입\n|로그인\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n기사검색\n\n\n2024\n10.28\n월요일 \n\n\n\n\n\n\n\n\n\n\n\n\n\n뉴스\n\n\n행정/법률의원/병원학술/학회의대/전공의제약‧바이오의료기기/IT간호치과약국/유통월드뉴스한방e-談 \n\n\n\n\n\n\n\n\n\n\n\n\n\n금주의 검색어\n\n\n\n\n1\n코로나19\n\n2\nK-방역\n\n3\n대한의사협회\n\n4\n간호법\n\n5\n삼성서울병원\n \n\n\n\n\n6\n비대면\n\n7\n응급의료\n\n8\n종근당\n\n9\n재택치료\n\n10\n의료법인',
 'metadata': {'source': 'https://www.dailymedi.com/news/news_view.php?wr_id=917206',
  'title': '유트로핀펜주·시네츄라·레켐비·펜타닐',
  'description': '\ufeff\ufeff\ufeff\ufeff[양보혜·구교윤 기자]\xa0유트로핀펜주를 비롯해 시네츄라,\xa0레켐비, 펜터민 및 에토미데이트 등.이는 국회 보건복지위원회가 지난 10일 진행한 식품의약품안전처 국정감사장을 뜨겁게 달군 의약품들이다. 화두가 된 이들 의약품을 통해 2024년 국감 이슈를 정리해봤다.\xa0먼저 유트로핀펜주는 국감장에 실제 등장했다. 김남희 더불어민주당 의원이 제품을 들고 나와 무분별하게 처방되고 있는 성장호르몬 주사에 대한 식약처의 부실한 관리 문제를 지적했다.\xa0김 의원은 ',
  'language': 'ko'}}

In [5]:
data

[{'page_content': '유트로핀펜주·시네츄라·레켐비·펜타닐\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n광고안내\n|회원가입\n|로그인\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n기사검색\n\n\n2024\n10.28\n월요일 \n\n\n\n\n\n\n\n\n\n\n\n\n\n뉴스\n\n\n행정/법률의원/병원학술/학회의대/전공의제약‧바이오의료기기/IT간호치과약국/유통월드뉴스한방e-談 \n\n\n\n\n\n\n\n\n\n\n\n\n\n금주의 검색어\n\n\n\n\n1\n코로나19\n\n2\nK-방역\n\n3\n대한의사협회\n\n4\n간호법\n\n5\n삼성서울병원\n \n\n\n\n\n6\n비대면\n\n7\n응급의료\n\n8\n종근당\n\n9\n재택치료\n\n10\n의료법인',
  'metadata': {'source': 'https://www.dailymedi.com/news/news_view.php?wr_id=917206',
   'title': '유트로핀펜주·시네츄라·레켐비·펜타닐',
   'description': '\ufeff\ufeff\ufeff\ufeff[양보혜·구교윤 기자]\xa0유트로핀펜주를 비롯해 시네츄라,\xa0레켐비, 펜터민 및 에토미데이트 등.이는 국회 보건복지위원회가 지난 10일 진행한 식품의약품안전처 국정감사장을 뜨겁게 달군 의약품들이다. 화두가 된 이들 의약품을 통해 2024년 국감 이슈를 정리해봤다.\xa0먼저 유트로핀펜주는 국감장에 실제 등장했다. 김남희 더불어민주당 의원이 제품을 들고 나와 무분별하게 처방되고 있는 성장호르몬 주사에 대한 식약처의 부실한 관리 문제를 지적했다.\xa0김 의원은 ',
   'language': 'ko'}},
 {'page_content': '8\n종근당\n\n9\n재택치료\n\n10\n의료법인\n \n\n\n\n\n\n\n\n\n\n\n\n\n뉴스\n행정/법률의원/병